In [1]:
import pandas as pd
from scipy.stats import zscore, f_oneway, kruskal

import sys
sys.path.append("../src")
import numpy as np
import nltk
from nltk.tree import Tree
import matplotlib.pyplot as plt
import pandas as pd
from iparse_vq import IParser

In [2]:
# gpn_vqtest4_labeled_16_back_test_cuda_8540884_dev=92.24.pt
# gpn_vqtest4_labeled_16_uni_cuda_8523872_dev=91.02.pt
iparse = IParser("../models/gpn_vqtest4_labeled_16_back_test_cuda_8540884_dev=92.24.pt")

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
def read_and_normalize(csv = 'tracking_data/selfpacedreading.RT.txt', var='RT'):
    reading = pd.read_csv(csv, delimiter = '\t')
    reading['zscore'] = reading.groupby(['subj_nr'], as_index=False)[var].transform(lambda x : zscore(x,ddof=1))
    return reading

# reads a csv data file then adds a column for the zscore of each specific speaker
reading = read_and_normalize()
display(reading.head(5))
reading.groupby('subj_nr').agg({'zscore': [np.mean, np.var]}).head()

,subj_nr,sent_nr,sent_pos,correct,answer_time,word_pos,word,RT,zscore
0,1,2,12,c,3630.0,1,Billy,376,1.314830
1,1,2,12,c,3630.0,2,wrote,364,1.181588
2,1,2,12,c,3630.0,3,on,394,1.514693
3,1,2,12,c,3630.0,4,the,353,1.059449
4,1,2,12,c,3630.0,5,envelope.,354,1.070552


zscore     
                 mean  var
subj_nr                   
1        1.111922e-16  1.0
2       -3.621307e-16  1.0
3        1.685706e-16  1.0
4        9.949097e-17  1.0
5        1.617035e-16  1.0

In [4]:
def combine_and_separate_periodt(lst):
    s = ' '.join(lst)
    return '{} .'.format(s[:-1])

def combine_sentences(reading):
    sentences = reading.sort_values(['sent_nr', 'word_pos']).groupby(['sent_nr', 'word_pos'], as_index=False).first()[['sent_nr', 'word_pos', 'word']]
    combined_sentences = sentences.groupby('sent_nr', as_index=False).agg({'word': combine_and_separate_periodt, 'word_pos':list})
    return combined_sentences

# creates a table of each sentence (to input into the parser)
combined = combine_sentences(reading)
combined

,sent_nr,word,word_pos
0,1,Anne lost control and laughed .,"[1, 2, 3, 4, 5]"
1,2,Billy wrote on the envelope .,"[1, 2, 3, 4, 5]"
2,3,He called over his shoulder .,"[1, 2, 3, 4, 5]"
3,4,He stayed against the wall .,"[1, 2, 3, 4, 5]"
4,5,Helen ran to the toilet .,"[1, 2, 3, 4, 5]"
...,...,...,...
356,357,Simon lay staring at the ceiling again with hi...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
357,358,Paula tried to relax back into the pillow to d...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
358,359,He kept his stock books neat and checked them ...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
359,360,Tom walked into the visiting room glad that hi...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [5]:
def label_sentences(iparse, combined_sentences):
    t = iparse.parse_batch(combined_sentences['word'])
    cats = []
    for tree, cat in t:
        cats.append(cat)
    return combined_sentences.assign(tags= cats)

# adds the tags to each sentence (including final period)
combined = label_sentences(iparse, combined)
combined

/Users/thomas/anaconda3/lib/python3.8/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


,sent_nr,word,word_pos,tags
0,1,Anne lost control and laughed .,"[1, 2, 3, 4, 5]","[6, 3, 14, 8, 9, 4]"
1,2,Billy wrote on the envelope .,"[1, 2, 3, 4, 5]","[6, 3, 12, 14, 11, 4]"
2,3,He called over his shoulder .,"[1, 2, 3, 4, 5]","[6, 3, 10, 14, 11, 4]"
3,4,He stayed against the wall .,"[1, 2, 3, 4, 5]","[6, 3, 12, 14, 11, 4]"
4,5,Helen ran to the toilet .,"[1, 2, 3, 4, 5]","[6, 3, 12, 14, 11, 4]"
...,...,...,...,...
356,357,Simon lay staring at the ceiling again with hi...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[6, 3, 3, 12, 14, 11, 1, 1, 14, 6, 12, 0, 2, 4..."
357,358,Paula tried to relax back into the pillow to d...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[6, 3, 12, 3, 5, 1, 14, 11, 1, 3, 5, 14, 4, 12..."
358,359,He kept his stock books neat and checked them ...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[6, 3, 14, 2, 2, 5, 4, 9, 14, 15, 14, 8, 15, 3..."
359,360,Tom walked into the visiting room glad that hi...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[6, 3, 12, 14, 11, 11, 1, 1, 15, 11, 9, 9, 14,..."


In [6]:
def uncombine_sentence(reading, combined_sentences, use_pairs=False):
    dict_lists = []
    for i, row in combined_sentences.iterrows():
        prev = ''
        for word_pos, tag in zip(row['word_pos'], row['tags']):
            tag2 = tag
            if use_pairs:
                tag2 = '{}-{}'.format(prev, tag)
                prev = tag
            dict_lists.append({'sent_nr': row['sent_nr'], 'word_pos':word_pos, 'tag': tag2})
    tags_for_words = pd.DataFrame(dict_lists)
    reading_with_tags = reading.merge(tags_for_words, on=['sent_nr', 'word_pos'])
    return reading_with_tags

# shows tag (or tag pair) for each word as in the readings table
r_with_tags = uncombine_sentence(reading, combined, use_pairs=False)
r_with_tags

,subj_nr,sent_nr,sent_pos,correct,answer_time,word_pos,word,RT,zscore,tag
0,1,2,12,c,3630.0,1,Billy,376,1.314830,6
1,2,2,179,c,3301.0,1,Billy,242,-0.220132,6
2,3,2,115,c,1822.0,1,Billy,329,0.037237,6
3,6,2,43,c,2550.0,1,Billy,208,-0.449477,6
4,8,2,87,e,3662.0,1,Billy,434,0.053585,6
...,...,...,...,...,...,...,...,...,...,...
353579,111,329,63,-,NaN,24,jacket.,694,1.989887,11
353580,112,329,4,-,NaN,24,jacket.,320,-0.174806,11
353581,114,329,233,-,NaN,24,jacket.,249,0.474962,11
353582,115,329,143,-,NaN,24,jacket.,195,-0.439994,11


In [7]:
def return_zscores(reading_with_tags):
    return reading_with_tags.groupby('tag').agg({'zscore': list}).sort_values('tag')['zscore'].tolist()

# finds the z-scores assigned to each tag
zscores_list = return_zscores(r_with_tags)
print('Categories: {} | List lengths: {}'.format(len(zscores_list), [len(lst) for lst in zscores_list]))

Categories: 16 | List lengths: [4748, 15077, 10061, 52395, 11347, 15537, 45112, 212, 16481, 23201, 11448, 30716, 37893, 1775, 56833, 20748]


In [8]:
# composes the above cells into a single function
# the combined argument lets us avoid recalculating the table of sentences and tags
#     because all the stimuli were the same
# supported stat tests are anova (assumes homoscedastic and normal), or kruskal (no assumption)

def read_csv_and_check(csv = 'tracking_data/selfpacedreading.RT.txt', var='RT', 
                       use_pairs=False, combined=None, iparse=iparse, test_type='kruskal'):
    r = read_and_normalize(csv = csv, var=var)
    if combined is None:
        combined = combine_sentences(r)
        combined = label_sentences(iparse, combined)
    r_with_tags = uncombine_sentence(r, combined, use_pairs)
    zscores_list = return_zscores(r_with_tags)
    return {'kruskal':kruskal, 'anova':f_oneway}[test_type](*zscores_list)

In [9]:
for csv, var in [('tracking_data/selfpacedreading.RT.txt', 'RT'), 
                 ('tracking_data/eyetracking.RT.txt', 'RTfirstfix'),
                ('tracking_data/eyetracking.RT.txt', 'RTfirstpass')]:
    for use_pairs in [True, False]:
        print(csv, var, 'Using Pairs: {}'.format(use_pairs))
        for test_type in ['kruskal', 'anova']:
            print(read_csv_and_check(csv = csv, var=var, use_pairs=use_pairs, combined=combined, test_type=test_type))
        print()


tracking_data/selfpacedreading.RT.txt RT Using Pairs: True
KruskalResult(statistic=855.9716188055553, pvalue=1.1083057268270747e-80)
F_onewayResult(statistic=4.731179203249096, pvalue=5.816226258542519e-99)

tracking_data/selfpacedreading.RT.txt RT Using Pairs: False
KruskalResult(statistic=170.5319705940634, pvalue=1.9114255736711924e-28)
F_onewayResult(statistic=16.76623331554992, pvalue=6.364987522059838e-45)

tracking_data/eyetracking.RT.txt RTfirstfix Using Pairs: True
KruskalResult(statistic=5290.784495921937, pvalue=0.0)
F_onewayResult(statistic=33.07406278334002, pvalue=0.0)

tracking_data/eyetracking.RT.txt RTfirstfix Using Pairs: False
KruskalResult(statistic=2897.857280317953, pvalue=0.0)
F_onewayResult(statistic=179.8326014101079, pvalue=0.0)

tracking_data/eyetracking.RT.txt RTfirstpass Using Pairs: True
KruskalResult(statistic=6421.3949022303095, pvalue=0.0)
F_onewayResult(statistic=37.962368725272825, pvalue=0.0)

tracking_data/eyetracking.RT.txt RTfirstpass Using Pairs:

In [10]:
# statistical tests between each tag
# useful for analysis?

from itertools import combinations
for cat1, cat2 in combinations(range(len(zscores_list)), 2):
    print(cat1, cat2, f_oneway(zscores_list[cat1], zscores_list[cat2]).pvalue)

0 1 0.002085717594829855
0 2 0.0036902037401918475
0 3 0.17013521399208906
0 4 0.0005628708084244676
0 5 0.009418407693087619
0 6 9.297397567774054e-07
0 7 2.548306784746384e-06
0 8 0.002843329308641143
0 9 0.29451631271144224
0 10 0.0001621268994474508
0 11 0.00691371493108525
0 12 0.11595086427962686
0 13 0.0452900704547211
0 14 0.15810574422711332
0 15 0.3500411975980131
1 2 0.9600838648463219
1 3 0.00024053218567642016
1 4 0.7627397816654033
1 5 0.2773424737477953
1 6 0.0014160577174169241
1 7 1.1354823915753824e-05
1 8 0.7492708442793374
1 9 0.0002727738597459031
1 10 0.11900003146847399
1 11 0.22049634270223947
1 12 0.0003835203022494737
1 13 0.8526676074908575
1 14 4.861144988321127e-05
1 15 0.00013480005012902384
2 3 0.0014446810824143396
2 4 0.8228538671990842
2 5 0.30538493267004396
2 6 0.00833380060823182
2 7 2.858745358299172e-05
2 8 0.7374731595867317
2 9 0.0011392714900028014
2 10 0.18102342936550758
2 11 0.26280031262934395
2 12 0.0016955715690579473
2 13 0.8789309022235